In [1]:
import tensorflow

In [242]:
# REQUIREMENTS
# Keras==2.2.0
# numpy==1.13.3
# tensorflow==1.7.0
# tensorflow_hub==0.1.1

In [2]:
print(tensorflow.__version__)

1.7.0


In [136]:
from IPython.core.debugger import set_trace



import pandas as pd
import numpy as np
import os
import time


In [137]:
SENTIMENT_LABELLED_DATA_FILEPATH = '..\Data\Sentiment Analysis Dataset.csv'

In [138]:
txt_data = pd.read_csv('..\Data\Sentiment Analysis Dataset.csv' , sep='|', names=['col1'])

#split data into columns with ','
txt_data = txt_data.col1.str.split(',',  3, expand=True)
txt_data.columns = list(txt_data.iloc[0])
txt_data = txt_data.drop(0)
txt_data.index = np.subtract(txt_data.index, 1)
x = txt_data.groupby('Sentiment')
l=[x.get_group(i)['SentimentText'] for i in x.groups]

neg = l[0]
pos = l[1]

In [209]:
dataset = pd.concat([txt_data['SentimentText'], txt_data['Sentiment']], axis = 1).sample(frac=1).reset_index(drop=True)
dataset.columns = ['text', 'target']
dataset = dataset.sample(frac=1)
train = dataset[:100000]
test = dataset[100000:120000]

In [211]:
train

,text,target
925644,@maryNUKElover into the nightlife by: cyndi la...,1
666446,@undagroundmilli (sigh) i don't know if there...,0
539555,@sleeprun Hmmm that's perceptive of you Why d...,1
309939,Okay I'm doing that thing where I rethink ever...,0
1189307,@rickoshea cool bath. or shower. rinse repeat,1
455327,is happy I got an A in my other class last ter...,0
999024,"""Done with most of my work, just some freaky p...",0
1344253,"""Yes i arrived home, felling Bad. I want you...",0
562180,i want to go to the show tonight next weekend...,0
1519037,can't upload my pic,0


In [222]:
from collections import Counter

# Count unique words
def counter_word(text):
    count = Counter()
    for i in text.values:
        for word in i.split():
            count[word] += 1
    return count

In [223]:
text = train.text

counter = counter_word(text)

In [224]:
num_words = len(counter)

# Max number of words in a sequence
max_length = 50

In [225]:
train_size = int(train.shape[0] * 0.8)

train_sentences = train.text[:train_size]
train_labels = train.target[:train_size]

test_sentences = train.text[train_size:]
test_labels = train.target[train_size:]

In [226]:
from keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer(num_words=num_words)
tokenizer.fit_on_texts(train_sentences)

In [227]:
word_index = tokenizer.word_index

In [228]:
from keras.preprocessing.sequence import pad_sequences
train_sequences = tokenizer.texts_to_sequences(train_sentences)
train_padded = pad_sequences(
    train_sequences, maxlen=max_length, padding="post", truncating="post"
)
test_sequences = tokenizer.texts_to_sequences(test_sentences)
test_padded = pad_sequences(
    test_sequences, maxlen=max_length, padding="post", truncating="post"
)

In [229]:
word_to_id = word_index
(x_train, y_train), (x_test, y_test) = (train_padded,train_labels.astype(int).values), (test_padded,test_labels.astype(int).values)

In [230]:
x_train.shape[0]

80000

In [231]:

from keras.preprocessing import sequence
from keras.datasets import imdb

def load_data(max_sequence_length):
    print('Loading data...')
#     (x_train, y_train), (x_test, y_test) = imdb.load_data()
    (x_train, y_train), (x_test, y_test)
    print(x_train)
    print(len(x_train), 'train sequences')
    print(len(x_test), 'test sequences')

    print('Pad sequences (samples x time)')
    x_train = sequence.pad_sequences(x_train, maxlen=max_sequence_length, padding='post', truncating='post')
    x_test = sequence.pad_sequences(x_test, maxlen=max_sequence_length, padding='post', truncating='post')
    print(x_train)
    print('x_train shape:', x_train.shape)
    print('x_test shape:', x_test.shape)
    return (x_train, y_train), (x_test, y_test)

def get_idx2word(word_to_id):
    INDEX_FROM = 3 # word index offset

#     word_to_id = imdb.get_word_index()
    word_to_id = {k:(v+INDEX_FROM) for k,v in word_to_id.items()}
    word_to_id["<PAD>"] = 0
    word_to_id["<START>"] = 1
    word_to_id["<UNK>"] = 2

    idx2word = {value:key for key,value in word_to_id.items()}
    return idx2word

In [232]:
#load_data(50)

In [233]:
from keras import backend as K
from keras.engine.topology import Layer
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np

class ELMoEmbedding(Layer):

    def __init__(self, idx2word, output_mode="lstm_outputs1", trainable=True, **kwargs):
        assert output_mode in ["default", "word_emb", "lstm_outputs1", "lstm_outputs2", "elmo"]
        assert trainable in [True, False]
        self.idx2word = idx2word
        self.output_mode = output_mode
        self.trainable = trainable
        self.max_length = None
        self.word_mapping = None
        self.lookup_table = None
        self.elmo_model = None
        self.embedding = None
        super(ELMoEmbedding, self).__init__(**kwargs)

    def build(self, input_shape):
        self.max_length = input_shape[1]
        self.word_mapping = [x[1] for x in sorted(self.idx2word.items(), key=lambda x: x[0])]
        self.lookup_table = tf.contrib.lookup.index_to_string_table_from_tensor(self.word_mapping, default_value="<UNK>")
        self.lookup_table.init.run(session=K.get_session())
        self.elmo_model = hub.Module("./elmo", trainable=self.trainable)
        #self.elmo_model = hub.Module("https://tfhub.dev/google/elmo/3", trainable=self.trainable)
        super(ELMoEmbedding, self).build(input_shape)

    def call(self, x):
        x = tf.cast(x, dtype=tf.int64)
        sequence_lengths = tf.cast(tf.count_nonzero(x, axis=1), dtype=tf.int32)
        strings = self.lookup_table.lookup(x)
        inputs = {
            "tokens": strings,
            "sequence_len": sequence_lengths
        }
        return self.elmo_model(inputs, signature="tokens", as_dict=True)[self.output_mode]

    def compute_output_shape(self, input_shape):
        if self.output_mode == "default":
            return (input_shape[0], 1024)
        if self.output_mode == "word_emb":
            return (input_shape[0], self.max_length, 512)
        if self.output_mode == "lstm_outputs1":
            return (input_shape[0], self.max_length, 1024)
        if self.output_mode == "lstm_outputs2":
            return (input_shape[0], self.max_length, 1024)
        if self.output_mode == "elmo":
            return (input_shape[0], self.max_length, 1024)

    def get_config(self):
        config = {
            'idx2word': self.idx2word,
            'output_mode': self.output_mode 
        }
        return list(config.items())

In [236]:
import tensorflow as tf
from keras.models import Model
from keras.layers import *
from keras.optimizers import Adam

#MAX_SEQUENCE_LENGTH = 100

#(x_train, y_train), (x_test, y_test) = load_data(max_sequence_length=MAX_SEQUENCE_LENGTH)
idx2word = get_idx2word(word_index)

sentence_input = Input(shape=(x_train.shape[1],), dtype=tf.int64)
sentence_embedding = ELMoEmbedding(idx2word=idx2word)(sentence_input) # These two are interchangeable
#dropout = Dropout(0.1)(sentence_embedding)
#hidden = Dense(100, activation='relu')(sentence_embedding)
hidden = LSTM(100, dropout=0.1)(sentence_embedding)
output = Dense(1, activation='sigmoid')(hidden)

model2 = Model(inputs=sentence_input, outputs=output)
model2.compile(loss='binary_crossentropy', optimizer=Adam(lr=3e-4), metrics=['accuracy'])
model2.summary()

#model.fit(x_train, y_train, batch_size=2, epochs=5, validation_data=(x_test, y_test))

INFO:tensorflow:Initialize variable module_29/aggregation/scaling:0 from checkpoint b'./elmo\\variables\\variables' with aggregation/scaling
INFO:tensorflow:Initialize variable module_29/aggregation/weights:0 from checkpoint b'./elmo\\variables\\variables' with aggregation/weights
INFO:tensorflow:Initialize variable module_29/bilm/CNN/W_cnn_0:0 from checkpoint b'./elmo\\variables\\variables' with bilm/CNN/W_cnn_0
INFO:tensorflow:Initialize variable module_29/bilm/CNN/W_cnn_1:0 from checkpoint b'./elmo\\variables\\variables' with bilm/CNN/W_cnn_1
INFO:tensorflow:Initialize variable module_29/bilm/CNN/W_cnn_2:0 from checkpoint b'./elmo\\variables\\variables' with bilm/CNN/W_cnn_2
INFO:tensorflow:Initialize variable module_29/bilm/CNN/W_cnn_3:0 from checkpoint b'./elmo\\variables\\variables' with bilm/CNN/W_cnn_3
INFO:tensorflow:Initialize variable module_29/bilm/CNN/W_cnn_4:0 from checkpoint b'./elmo\\variables\\variables' with bilm/CNN/W_cnn_4
INFO:tensorflow:Initialize variable module_

In [196]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_21 (InputLayer)        (None, 50)                0         
_________________________________________________________________
el_mo_embedding_22 (ELMoEmbe (None, 1024)              0         
_________________________________________________________________
dense_22 (Dense)             (None, 100)               102500    
_________________________________________________________________
dense_23 (Dense)             (None, 1)                 101       
Total params: 102,601
Trainable params: 102,601
Non-trainable params: 0
_________________________________________________________________


In [237]:
model2.fit(x_train, y_train, batch_size=128, epochs=5, validation_data=(x_test, y_test))


Train on 80000 samples, validate on 20000 samples
Epoch 1/5
80000/80000 [==============================] - 3966s 50ms/step - loss: 0.6204 - acc: 0.6456 - val_loss: 0.5719 - val_acc: 0.7060
Epoch 2/5
80000/80000 [==============================] - 3842s 48ms/step - loss: 0.5467 - acc: 0.7215 - val_loss: 0.5515 - val_acc: 0.7229
Epoch 3/5
80000/80000 [==============================] - 3866s 48ms/step - loss: 0.5002 - acc: 0.7546 - val_loss: 0.5251 - val_acc: 0.7339
Epoch 4/5
80000/80000 [==============================] - 3748s 47ms/step - loss: 0.4622 - acc: 0.7804 - val_loss: 0.5304 - val_acc: 0.7366
Epoch 5/5
80000/80000 [==============================] - 3755s 47ms/step - loss: 0.4272 - acc: 0.8016 - val_loss: 0.5208 - val_acc: 0.7486


In [95]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_17 (InputLayer)        (None, 50)                0         
_________________________________________________________________
el_mo_embedding_18 (ELMoEmbe (None, 1024)              0         
_________________________________________________________________
dropout_7 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_14 (Dense)             (None, 50)                51250     
_________________________________________________________________
dense_15 (Dense)             (None, 1)                 51        
Total params: 51,301
Trainable params: 51,301
Non-trainable params: 0
_________________________________________________________________


In [241]:
MODEL_OUTPUT_FILEPATH = 'model_elmo'
model2.save(MODEL_OUTPUT_FILEPATH,save_format="tf")

TypeError: save() got an unexpected keyword argument 'save_format'

In [113]:
(x_train, y_train), (x_test, y_test) = load_data(max_sequence_length=50)

Loading data...
[ list([1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 22665, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 21631, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 19193, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 5244, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 10311, 8, 4, 107, 117, 5952, 15, 256, 4, 31050, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 12118, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 7486, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 5535, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 16, 5345, 19, 178, 32])
 list([1, 19

In [125]:
x_test.size

100000

In [123]:
y_pred_keras

array([[ 0.46481028],
       [ 0.55957186],
       [ 0.58679086]], dtype=float32)

In [122]:
y_pred_keras = model.predict(x_test[:3])

In [239]:
y_pred_keras = model2.predict(x_test).ravel()

elmo_roc = pd.DataFrame([test_labels.values, y_pred_keras]).T
elmo_roc.columns = ['test_labels', 'y_pred_keras']
elmo_roc.to_csv('elmo_roc5.csv')

In [189]:
y_pred_keras = model.predict(x_train[5000:]).ravel()

elmo_roc = pd.DataFrame([y_train[5000:], y_pred_keras]).T
elmo_roc.columns = ['test_labels', 'y_pred_keras']
elmo_roc.to_csv('elmo_roc3.csv')

In [190]:
elmo_roc

,test_labels,y_pred_keras
0,0.0,0.255155
1,1.0,0.689691
2,0.0,0.380691
3,0.0,0.148736
4,1.0,0.882411
5,1.0,0.755543
6,0.0,0.471286
7,1.0,0.149354
8,1.0,0.768143
9,1.0,0.478752
